In [33]:
import findspark
findspark.init()

In [34]:
# Create SparkSession and import all required packages
from pyspark.sql import SparkSession,types

spark = SparkSession.builder.master("local").appName('Json File')\
                    .getOrCreate()

filepath = "file:///C:/Users/venka/PycharmProjects/pythonProject/dataset/"

In [35]:
#read multiline JSON file
input_df=spark.read.json(filepath + 'IntMergeJsonInput2.json', multiLine=True)

In [36]:
input_df.printSchema()

root
 |-- Education: struct (nullable = true)
 |    |-- Qualification: string (nullable = true)
 |    |-- year: long (nullable = true)
 |-- name: string (nullable = true)



In [37]:
input_df.show()

+----------+-------+
| Education|   name|
+----------+-------+
|{BE, 2011}| Clarke|
|{BE, 2010}|Michael|
+----------+-------+



In [38]:
#read multiline JSON file
input_df1=spark.read.json(filepath + 'IntMergeJsonInput1.json', multiLine=True)

In [39]:
input_df1.printSchema()

root
 |-- Education: struct (nullable = true)
 |    |-- Age: long (nullable = true)
 |    |-- Qualification: string (nullable = true)
 |    |-- year: long (nullable = true)
 |-- name: string (nullable = true)



In [40]:
input_df1.schema

StructType([StructField('Education', StructType([StructField('Age', LongType(), True), StructField('Qualification', StringType(), True), StructField('year', LongType(), True)]), True), StructField('name', StringType(), True)])

In [41]:
from pyspark.sql.types import *
from pyspark.sql.functions import col,lit,struct

def flatten_struct(schema, prefix=""):
    result = []
    for elem in schema:
        if isinstance(elem.dataType, StructType):
            result += flatten_struct(elem.dataType, prefix + elem.name + ".")
        else:
            result.append(prefix + elem.name)
    return result

l1=flatten_struct(input_df.schema)
l2=flatten_struct(input_df1.schema)

In [42]:
l1

['Education.Qualification', 'Education.year', 'name']

In [43]:
list1=[]
list2=[]
for i in l1:
    list1.append(str(i))
    
for i in l2:
    list2.append(str(i))
    

In [44]:
chk=set(list2)-set(list1)
chk

{'Education.Age'}

In [45]:
type(list1)

list

In [48]:
from pyspark.sql.types import *
chk=set(list2)-set(list1)
for i in chk:
    if i.find("."):
        colm=i.split(".")[0]
        colm_new=i.split(".")[1]
        s_fields = input_df.schema[colm].dataType.names
        s_type=input_df1.schema[colm].dataType[colm_new].dataType
        
        input_df.show()
        in_df=input_df.withColumn(colm,
                            struct(*([col(colm)[c].alias(c) for c in s_fields] +
                                     [lit("null").cast(s_type).alias(colm_new)]
                                     ))
                                 )
        in_df.show()
        s_fields = sorted(in_df.schema[colm].dataType.names)
        
        
        in_df=in_df.withColumn(colm,
                            struct(*([col(colm)[c].alias(c) for c in s_fields] ))
                                 )
        
        
        


+----------+-------+
| Education|   name|
+----------+-------+
|{BE, 2011}| Clarke|
|{BE, 2010}|Michael|
+----------+-------+

+----------------+-------+
|       Education|   name|
+----------------+-------+
|{BE, 2011, NULL}| Clarke|
|{BE, 2010, NULL}|Michael|
+----------------+-------+



In [49]:
in_df.printSchema()

root
 |-- Education: struct (nullable = false)
 |    |-- Age: long (nullable = true)
 |    |-- Qualification: string (nullable = true)
 |    |-- year: long (nullable = true)
 |-- name: string (nullable = true)



In [50]:
out_df=in_df.union(input_df1)

In [51]:
out_df.show()

+----------------+-------+
|       Education|   name|
+----------------+-------+
|{NULL, BE, 2011}| Clarke|
|{NULL, BE, 2010}|Michael|
|{28, BCOM, 2013}|   AZAR|
|  {35, BE, 2010}|   CHIN|
+----------------+-------+

